In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Aug 18 18:44:12 2023

@author: adelrioc
"""
###########
# Imports #
###########

# importing libraries
import matplotlib.pyplot as plt
import copy
# importing algorithms
from BFGS_multistart import* 
#from PSO import* 
#from Stochastic_local_search import* 
#from Cuadratic_opt import*
from Scipy_opt_algs import*
# from BO_NpScpy_1var_SSAdq import*
# from BO_NpScpy_1var import*
# from BO_NpScpy_2var_SSAdq import*
# from BO_NpScpy_3var import*
# from BO_NpScpy_3var_SSAdq import*
# from BO_NpScpy import*
# from BO_NpScpy_v2 import*
# from BO_NpScpy_v2_worstMean import*
# from BO_NpScpy_v2_Qmean import*
# from BO_NpScpy_v2_Qmean_worstMean import*
# from BO_NpScpy_v3 import*
# from BO_NpScpy_v2_Ax import*
# from BO_NpScpy_TURBO import*
# from BO_NpScpy_TURBO_Ax import*
# from BO_NpScpy_TR import*
# importing test functions
from test_function_antonio import* 
# importing utilities
# from plot_utils_v2 import*
from ScikitQuant_opt_algs import*
from pySOT_opt_algs import*
#from ENTMOOT_opt import*
from Cuadratic_opt_v2 import*

In [2]:
##########################
# Algorithms & Functions #
##########################

algorithms_test = [#BFGS_gs, 
                   #LS_QM, 
                   #SS_alg, 
                   #opt_Powell, 
                   #opt_DE, 
                   #opt_Basinhopping,
                   #opt_Direct,
                   opt_SOP, #
                   opt_SRBF,#
                   opt_DYCORS,#
                   #opt_ENTMOOT,#
                   LS_QM_v2,
                #    opt_COBYLA, 
                   #opt_NelderMead
                   #BO_np_scipy_1var,
                   #BO_np_scipy_1var_SSAdq,
                   #BO_np_scipy,
                   #BO_np_scipy_2var_SSAdq,
                   #BO_np_scipy_3var,
                   #BO_np_scipy_3var_SSAdq,
                   #BO_np_scipy_2M,
                   #BO_np_scipy_2Ax,
                   #BO_np_scipy_3,
                   #BO_np_scipy_2Qmean_wMean,
                   #BO_np_scipy_2Qmean,                   
                   #BO_np_scipy_2wMean,                   
                   #BO_np_scipy_2,      # best
                   #BO_np_scipy_turbo_Ax,
                   #BO_np_scipy_turbo
                   #BO_np_scipy_TR
                   #random_search
                   opt_SnobFit,
                   opt_Bobyqa
                   ]
#functions_test  = ['1_norm','Ackely_f','Rosenbrock_f', 'Levy_f', 'Rastringin_f']
functions_test  = ['Rosenbrock_f', 'Rastringin_f', 'Levy_f', 'Ackely_f']

###########################
# Optimization parameters #
###########################

N_x_l        = [2,5,7]#[5, 7]#[10, 30] # [2, 5, 10]    # dimensions
start_       = [5,10,15]#[20, 25]#[25, 50] # [5, 10, 15]   # start counting
reps         = 5
f_eval_l     = [20,50,70]#[50, 70]#[100, 300] # [20, 50, 100] # function evaluations
test_results = {}

#############
# Dimension #
#############
for i_run in range(len(N_x_l)):
    N_x_                = N_x_l[i_run]
    f_eval_             = f_eval_l[i_run]
    dim_S               = 'D'+str(N_x_)
    test_results[dim_S] = {}
    # TODO change to -7,7
    bounds_             = np.array([[-10,10] for i in range(N_x_)]) # 10,10
    
    #############
    # Functions #
    #############
    for i_function in functions_test:
        print('===================== ',i_function,'D'+str(N_x_))
        test_results[dim_S][i_function]              = {}
        test_results[dim_S][i_function]['all means'] = {}
        test_results[dim_S][i_function]['all 95']    = {}
        test_results[dim_S][i_function]['all 05']    = {}
        all_f_                                       = []
        randShift_l                                  = np.random.uniform(-3,3, (reps,N_x_))
        
        
        ##############
        # Algorithms #
        ##############
        for i_algorithm in algorithms_test:
            print('== ',str(i_algorithm.__name__))
            test_results[dim_S][i_function][str(i_algorithm.__name__)] = []
            
            ###############
            # Repetitions #
            ###############
            for i_rep in range(reps):
                # random shift
                x_shift_ = randShift_l[i_rep,:].reshape((N_x_,1))
                # test function
                t_       = Test_function(i_function, N_x_, False, x_shift_, bounds_)
                # algorithm
                if str(i_algorithm.__name__) == 'opt_DYCORS':
                    i_algorithm(t_, N_x_, bounds_, f_eval_)
                elif str(i_algorithm.__name__) == 'opt_SRBF':
                    i_algorithm(t_, N_x_, bounds_, f_eval_)
                elif str(i_algorithm.__name__) == 'opt_SOP':
                    i_algorithm(t_, N_x_, bounds_, f_eval_)
                else:
                    i_algorithm(t_.eval, N_x_, bounds_, f_eval_)
                # post-processing
                t_.best_f_list()
                t_.pad_or_truncate(f_eval_)
                # store result
                test_results[dim_S][i_function][str(i_algorithm.__name__)].append(copy.deepcopy(t_.best_f_c))     
            # statistics from each algorithm for a function    
            l_   = np.array(test_results[dim_S][i_function][str(i_algorithm.__name__)])
            m_   = np.mean(l_, axis=0)
            q05_ = np.quantile(l_, 0.05, axis=0)
            q95_ = np.quantile(l_, 0.95, axis=0)
            test_results[dim_S][i_function]['all means'][str(i_algorithm.__name__)] = copy.deepcopy(m_)
            test_results[dim_S][i_function]['all 95'][str(i_algorithm.__name__)]    = copy.deepcopy(q05_)
            test_results[dim_S][i_function]['all 05'][str(i_algorithm.__name__)]    = copy.deepcopy(q95_)
            all_f_.append(copy.deepcopy(l_))
        # statistics for all algorithms for a function       
        test_results[dim_S][i_function]['mean']   = np.mean(all_f_, axis=(0,1))
        test_results[dim_S][i_function]['median'] = np.median(all_f_, axis=(0,1))
        test_results[dim_S][i_function]['q 0']    = np.max(all_f_, axis=(0,1))
        test_results[dim_S][i_function]['q 100']  = np.min(all_f_, axis=(0,1))
    
# import pickle
# #import _pickle as pickle 

# with open('results_algorithms_Plots.pkl', 'wb') as file:
#     file.write(pickle.dumps(test_results))

# # plot results
# folderplt_ = 'plots/Convergence_plots/'
# plot_performance(test_results, algorithms_test, functions_test, folderplt_, N_x_l)
            
# # https://stackoverflow.com/questions/19201290/how-to-save-a-dictionary-to-a-file
    
            


# set optimization function
#Ros = Test_function('Rosenbrock_f', N_x_, True)

#LS_QM(Ros.fun_test, N_x_, bounds_, 100)



#plt.plot(np.log(Ros.best_f_c))
#plt.show()
#plt.plot(np.log(Ros.best_f))
#plt.show()
#plt.plot(np.log(Ros.f_list_c))
#plt.show()
#plt.scatter( np.array(Ros.x_list)[:,0].flatten(), np.array(Ros.x_list)[:,1].flatten() )
#plt.show()

# test problems
#multimodal_tests = [Rastringin_f, Ackely_f]
#multimodal_name  = ['Rastringin_f', 'Ackely_f']
#Ill_conditioned  = [Rosenbrock_f, Levy_f]
#Ill_cond_name    = ['Rosenbrock_f','Levy_f']
#flat_shaped      = [Zakharov_f, Michalewicz_f]
#flat_name        = ['Zakharov_f', 'Michalewicz_f']

=====================  Rosenbrock_f D2
==  opt_SOP
==  opt_SRBF
==  opt_DYCORS
==  LS_QM_v2
iter  0 / 17  radius =  0.5
iter  1 / 17  radius =  0.25
iter  2 / 17  radius =  0.125
iter  3 / 17  radius =  0.0625
iter  4 / 17  radius =  0.125
iter  5 / 17  radius =  0.25
iter  6 / 17  radius =  0.125
iter  7 / 17  radius =  0.25
iter  8 / 17  radius =  0.125
iter  9 / 17  radius =  0.0625
iter  10 / 17  radius =  0.125
iter  11 / 17  radius =  0.0625
iter  12 / 17  radius =  0.03125
iter  13 / 17  radius =  0.0625
iter  14 / 17  radius =  0.03125
iter  15 / 17  radius =  0.0625
iter  16 / 17  radius =  0.03125
iter  0 / 17  radius =  0.5
iter  1 / 17  radius =  0.25
iter  2 / 17  radius =  0.125
iter  3 / 17  radius =  0.0625
iter  4 / 17  radius =  0.03125
iter  5 / 17  radius =  0.0625
iter  6 / 17  radius =  0.125
iter  7 / 17  radius =  0.25
iter  8 / 17  radius =  0.5
iter  9 / 17  radius =  0.25
iter  10 / 17  radius =  0.5


KeyboardInterrupt: 